# Aerodynamic Splatting

In [1]:
import numpy as np
from sympy import symbols, diff, Quaternion, exp, Matrix, lambdify, sin, cos, simplify
from sympy.physics.mechanics import *
from scipy.spatial.transform import Rotation
from IPython.display import display

## Create Symbolic Functions

In [2]:
x, y, z, u, v, w, sx, sy, sz, qw, qx, qy, qz, θ = symbols("x, y, z, u, v, w, s_x, s_y, s_z, q_w, q_x, q_y, q_z, θ")
xp, yp, zp = symbols("x_p, y_p, z_p")

quat = lambda q: Quaternion(*q) if isinstance(q, np.ndarray) else Quaternion.from_axis_angle([0, 0, 1], q)

position = np.array([x, y, z])
eval_at_position = np.array([xp, yp, zp])
velocity = np.array([u, v, w])
scale = np.array([sx, sy, sz])
orientation= np.array([qw, qx, qy, qz])
gaussian = lambda variance: exp(-0.5 * variance)
def velocity_eqn():
    quaternion = quat(orientation)
    rotation_matrix = quaternion.to_rotation_matrix().subs(quaternion.norm(), 1)
    rotation_matrix.simplify()
    dx = rotation_matrix @ (eval_at_position - position)
    s = dx / scale
    variance = np.sum(s**2)
    return velocity * gaussian(variance)
    
display(Matrix(velocity_eqn()))

Matrix([
[u*exp(-0.5*((-x + x_p)*(-2*q_w*q_y + 2*q_x*q_z) + (-y + y_p)*(2*q_w*q_x + 2*q_y*q_z) + (-z + z_p)*(-2*q_x**2 - 2*q_y**2 + 1))**2/s_z**2 - 0.5*((-x + x_p)*(2*q_w*q_z + 2*q_x*q_y) + (-y + y_p)*(-2*q_x**2 - 2*q_z**2 + 1) + (-z + z_p)*(-2*q_w*q_x + 2*q_y*q_z))**2/s_y**2 - 0.5*((-x + x_p)*(-2*q_y**2 - 2*q_z**2 + 1) + (-y + y_p)*(-2*q_w*q_z + 2*q_x*q_y) + (-z + z_p)*(2*q_w*q_y + 2*q_x*q_z))**2/s_x**2)],
[v*exp(-0.5*((-x + x_p)*(-2*q_w*q_y + 2*q_x*q_z) + (-y + y_p)*(2*q_w*q_x + 2*q_y*q_z) + (-z + z_p)*(-2*q_x**2 - 2*q_y**2 + 1))**2/s_z**2 - 0.5*((-x + x_p)*(2*q_w*q_z + 2*q_x*q_y) + (-y + y_p)*(-2*q_x**2 - 2*q_z**2 + 1) + (-z + z_p)*(-2*q_w*q_x + 2*q_y*q_z))**2/s_y**2 - 0.5*((-x + x_p)*(-2*q_y**2 - 2*q_z**2 + 1) + (-y + y_p)*(-2*q_w*q_z + 2*q_x*q_y) + (-z + z_p)*(2*q_w*q_y + 2*q_x*q_z))**2/s_x**2)],
[w*exp(-0.5*((-x + x_p)*(-2*q_w*q_y + 2*q_x*q_z) + (-y + y_p)*(2*q_w*q_x + 2*q_y*q_z) + (-z + z_p)*(-2*q_x**2 - 2*q_y**2 + 1))**2/s_z**2 - 0.5*((-x + x_p)*(2*q_w*q_z + 2*q_x*q_y) + (-y + 

## Create Splatting Class

In [3]:
class AeroSplat:
    position = np.zeros(2)
    velocity = np.zeros(2)
    scale = np.ones(2)
    orientation = 0 # np.array([0, 0, 0, 1])
    
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def __repr__(self):
        return f"AeroSplat(position={self.position}, velocity={self.velocity}, scale={self.scale}, orientation={self.orientation})"
    
    @property
    def quaternion(self):
        return quat(self.orientation)
    
    @property
    def rotation_matrix(self):
        R = self.quaternion.to_rotation_matrix()
        R.simplify()
        return R[:2, :2] if len(self.position) == 2 else R
    
    @property
    def scale_matrix(self):
        return np.diag(self.scale)
    
    def velocity_at(self, position):
        dx = self.rotation_matrix @ (position - self.position)
        s = dx / self.scale
        variance = np.sum(s**2)
        try:
            return self.velocity * np.exp(-0.5 * variance)
        except:
            return self.velocity * exp(-0.5 * variance)
    
    def differential_velocity_at(self, position):
        return np.zeros(3)
        

In [4]:
a = AeroSplat(
    position=np.array([x, y, z]), 
    velocity=np.array([u, v, w]),
    scale=np.array([sx, sy, sz]),
    orientation=np.array([qx, qy, qz, qw])
)
display(a)
display(a.quaternion)
display(a.rotation_matrix)
display(Matrix(a.velocity_at(np.array([xp, yp, zp]))))
#display(Matrix(a.velocity_at(np.ones(3))))

AeroSplat(position=[x y z], velocity=[u v w], scale=[s_x s_y s_z], orientation=[q_x q_y q_z q_w])

q_x + q_y*i + q_z*j + q_w*k

Matrix([
[(-q_w**2 + q_x**2 + q_y**2 - q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2),               2*(-q_w*q_x + q_y*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2),               2*(q_w*q_y + q_x*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2)],
[               2*(q_w*q_x + q_y*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2), (-q_w**2 + q_x**2 - q_y**2 + q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2),               2*(q_w*q_z - q_x*q_y)/(q_w**2 + q_x**2 + q_y**2 + q_z**2)],
[               2*(q_w*q_y - q_x*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2),                2*(q_w*q_z + q_x*q_y)/(q_w**2 + q_x**2 + q_y**2 + q_z**2), (q_w**2 + q_x**2 - q_y**2 - q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2)]])

Matrix([
[u*exp(-0.5*(2*(-x + x_p)*(q_w*q_y - q_x*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + 2*(-y + y_p)*(q_w*q_z + q_x*q_y)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + (-z + z_p)*(q_w**2 + q_x**2 - q_y**2 - q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2))**2/s_z**2 - 0.5*(2*(-x + x_p)*(q_w*q_x + q_y*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + (-y + y_p)*(-q_w**2 + q_x**2 - q_y**2 + q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + 2*(-z + z_p)*(q_w*q_z - q_x*q_y)/(q_w**2 + q_x**2 + q_y**2 + q_z**2))**2/s_y**2 - 0.5*((-x + x_p)*(-q_w**2 + q_x**2 + q_y**2 - q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + 2*(-y + y_p)*(-q_w*q_x + q_y*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + 2*(-z + z_p)*(q_w*q_y + q_x*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2))**2/s_x**2)],
[v*exp(-0.5*(2*(-x + x_p)*(q_w*q_y - q_x*q_z)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + 2*(-y + y_p)*(q_w*q_z + q_x*q_y)/(q_w**2 + q_x**2 + q_y**2 + q_z**2) + (-z + z_p)*(q_w**2 + q_x**2 - q_y**2 - q_z**2)/(q_w**2 + q_x**2 + q_y**2 + q_z**2))**2/s_z**2 -

In [5]:
class LineBoundary:
    point0 = np.zeros(3)
    point1 = np.zeros(3)
    
    def __init__(self, point0=np.zeros(3), point1=np.zeros(3)):
        self.point0 = point0
        self.point1 = point1
        
    def __repr__(self):
        return f"LineBoundary(point0={self.point0}, point1={self.point1})"

In [6]:
line = LineBoundary(point1=np.ones(3))

print(line)

LineBoundary(point0=[0. 0. 0.], point1=[1. 1. 1.])
